In [ ]:
%%bash
# Installs the latest dev build of TVM from PyPI. If you wish to build
# from source, see https://tvm.apache.org/docs/install/from_source.html
pip install apache-tvm --pre

How to Use TVM Pass Infra {#tutorial-use-pass-infra}
=========================

**Author**: [Zhi Chen](https://github.com/zhiics)

As the number of optimization passes increases in Relay/tir, it becomes
intractable to execute them and maintain their dependencies manually.
Therefore, we have introduced an infrastructure to manage the
optimization passes and make it applicable to different layers of the IR
in the TVM stack.

The optimizations of a Relay/tir program could be applied at various
granularity, namely function-level and module-level using
:py`tvm.relay.transform.FunctionPass`{.interpreted-text role="class"}/
:py`tvm.tir.transform.PrimFuncPass`{.interpreted-text role="class"} and
:py`tvm.transform.ModulePass`{.interpreted-text role="class"}
respectively. Or users can rely on
:py`tvm.transform.Sequential`{.interpreted-text role="class"} to apply a
sequence of passes on a Relay/tir program where the dependencies between
passes can be resolved by the pass infra. For more details about each
type of these passes, please refer to the `pass-infra`{.interpreted-text
role="ref"}

This tutorial mainly demonstrates how developers can use the pass infra
to perform a certain optimization and create an optimization pipeline
for a Relay program. The same approach can be used for tir as well.


In [ ]:
import numpy as np
import tvm
from tvm import te
import tvm.relay as relay

Create An Example Relay Program
===============================

First of all, we create a simple Relay program for the tutorial. This
program will be used by various optimizations of the examples in this
tutorial. Similarly, users can write a tir primitive function and apply
the tir passes.


In [ ]:
def example():
    shape = (1, 64, 54, 54)
    c_data = np.empty(shape).astype("float32")
    c = relay.const(c_data)
    weight = relay.var("weight", shape=(64, 64, 3, 3))
    x = relay.var("x", relay.TensorType((1, 64, 56, 56), "float32"))
    conv = relay.nn.conv2d(x, weight)
    y = relay.add(c, c)
    y = relay.multiply(y, relay.const(2, "float32"))
    y = relay.add(conv, y)
    z = relay.add(y, c)
    z1 = relay.add(y, c)
    z2 = relay.add(z, z1)
    return relay.Function([x, weight], z2)

Optimize the Program
====================

Now we would like to optimize the program. Relay features a host of
optimizations. We will select some of them to apply on this example
program.

There are multiple ways to optimize a Relay program. Below we will
provide examples for each of them.

Manually Apply Optimization Passes
----------------------------------


In [ ]:
# Let's first create a relay Module which contains one or multiple Relay
# functions for optimization.
f = example()
mod = tvm.IRModule.from_expr(f)

# Now we can apply constant folding on the module.
# fold_const here is a callback that doesn't take any parameters.
fold_const = relay.transform.FoldConstant()
# Then, we can invoke the pass on the given module. Note that the constant
# folding pass works at the function-level. That being said, each function in
# the module will be applied with the optimization. Users don't need to iterate
# through individual functions manually to apply this pass.
mod = fold_const(mod)
# We can see from the updated program that the constants are folded.
print(mod)

More optimizations can be applied in the similar manner. For instance,
we can eliminate the common expressions that used by [z]{.title-ref} and
[z1]{.title-ref}.


In [ ]:
mod = relay.transform.EliminateCommonSubexpr()(mod)
print(mod)

Some optimizations, such as fusion, are parametric as well. For example,
opt level 0 will not allow operators to be fused together. Users can
pass the [fuse\_opt\_level]{.title-ref} to enable this.


In [ ]:
mod = relay.transform.FuseOps(fuse_opt_level=0)(mod)

# We can observe that the optimized module contains functions that only have
# a signle primitive op.
print(mod)

Use Sequential to Apply a Sequence of Passes
============================================

Applying passes as above is actually tedious and it may require users to
have better understanding about the dependencies between them. For
example, fusion currently doesn\'t work well on let bindings. Therefore,
we would not be able to fuse operators that were fusable if
:py`relay.transform.ToANormalForm`{.interpreted-text role="func"} is
applied before fusion, as this pass generates let bindings for each
expression to canonicalize a Relay program.

Relay, hence, provides :py`tvm.transform.Sequential`{.interpreted-text
role="class"} to alleviate developers from handling these issues
explicitly by specifying the required passes of each pass and packing
them as a whole to execute. For example, the same passes can now be
applied using the sequential style as the following.
:py`tvm.transform.Sequential`{.interpreted-text role="class"} is similar
to
[torch.nn.sequential](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)
and
[mxnet.gluon.block](https://mxnet.apache.org/api/python/docs/_modules/mxnet/gluon/block.html).
For example, [torch.nn.sequential]{.title-ref} is used to contain a
sequence of PyTorch [Modules]{.title-ref} that will be added to build a
network. It focuses on the network layers. Instead, the
:py`tvm.transform.Sequential`{.interpreted-text role="class"} in our
pass infra works on the optimizing pass.


In [ ]:
# Now let's execute some passes through :py:class:`tvm.transform.Sequential`
f = example()
mod = tvm.IRModule.from_expr(f)
# Glob the interested passes.
seq = tvm.transform.Sequential(
    [
        relay.transform.FoldConstant(),
        relay.transform.EliminateCommonSubexpr(),
        relay.transform.FuseOps(fuse_opt_level=2),
    ]
)
mod1 = seq(mod)
print(mod1)

From the transformed Relay program, we can see that there are still two
identical addition operations. This is because `EliminateCommonSubexpr`
was not actually performed. The reason is because only the passes that
have optimization level less or equal to 2 will be executed by default
under :py`tvm.transform.Sequential`{.interpreted-text role="class"}. The
pass infra, however, provides a configuration interface for users to
customize the optimization level that they want to execute.


In [ ]:
with tvm.transform.PassContext(opt_level=3):
    mod2 = seq(mod)
print(mod2)

Now we can see that only one of the two identical additions is kept.

In addition, users can selectively disable some passes using the
[disabled\_pass]{.title-ref} config, which is similar to the
[-fno-xxx]{.title-ref} option used the general purpose compilers, such
as Clang and GCC. For example, we can disable EliminateCommonSubexpr as
following. The printed module will again show two identical addition
operations.


In [ ]:
with tvm.transform.PassContext(opt_level=3, disabled_pass=["EliminateCommonSubexpr"]):
    mod3 = seq(mod)
print(mod3)

Implement a Pass Using Python Decorator
=======================================

The next example illustrates how we can orchestrate a customized
optimization pipeline through the pass infra using Python decorators.
This functionality greatly eases the implementation of passes. For
example, users can simply define a decorated class to do function-level
optimizations as the following example shows.
[transform\_function]{.title-ref} wraps a class to replace all constants
with a multiple of [c]{.title-ref}. Later on, each function in a given
module will be visited and each constant in the function will be
replaced when we invoke the customized pass.


In [ ]:
@relay.transform.function_pass(opt_level=1)
class CustomPipeline:
    """Simple test function to replace one argument to another."""

    def __init__(self, multiplier):
        self.multiplier = multiplier

    # This function can define a pass.
    def transform_function(self, func, mod, ctx):
        obj = self

        class ReplaceConstant(tvm.relay.ExprMutator):
            def visit_constant(self, c):
                return relay.multiply(obj.multiplier, c)

        return ReplaceConstant().visit(func)


f = example()
mod = tvm.IRModule.from_expr(f)
custom_pass = CustomPipeline(multiplier=relay.const(3, "float32"))
assert custom_pass.info.name == "CustomPipeline"
mod3 = custom_pass(mod)
print(mod3)

Debug a Pass
============

TVM provides users a plug-and-play style debugging pass that print the
IR after a certain pass is done through a special pass (`PrintIR`) to
dump the IR of the whole module. A slightly modified version of the
sequential pass example could be like the following to enable IR dumping
for `FoldConstant` optimization.


In [ ]:
f = example()
mod = tvm.IRModule.from_expr(f)
seq = tvm.transform.Sequential(
    [
        relay.transform.FoldConstant(),
        tvm.transform.PrintIR(),
        relay.transform.EliminateCommonSubexpr(),
        relay.transform.FuseOps(),
    ]
)

By inserting the `PrintIR` pass after `FoldConstant`, the pass infra
will dump out the module IR when `FoldConstant` is done. Users can plug
in this pass after any pass they want to debug for viewing the
optimization effect.

There is a more flexible debugging mechanism. One can implement a
`PassInstrument` class to execute arbitrary code not only before and/or
after each pass but also at entering/exiting `PassContext`. See
`pass_instrument_cpp_backend`{.interpreted-text role="ref"} for more
details.

Here we use :py::func\`tvm.instrument.pass\_instrument\` decorator to
implement a PassInsturment class printing IR before execution of each
passes:


In [ ]:
@tvm.instrument.pass_instrument
class PrintIR:
    """Print the name of the pass, the IR, only before passes execute."""

    def run_before_pass(self, mod, info):
        print("Running pass: {}", info)
        print(mod)


with tvm.transform.PassContext(opt_level=3, instruments=[PrintIR()]):
    with tvm.target.Target("llvm"):
        # Perform the optimizations.
        mod = seq(mod)
print(mod)

print("done")

Summary
=======

This tutorial has covered how we can write and invoke passes in TVM more
conveniently using the pass infra. Different ways of invoking a pass are
also discussed. Using :py`tvm.transform.Sequential`{.interpreted-text
role="class"} can largely help users to ease the work of handling
multiple optimization passes and their dependencies. In addition, an
example is provided to illustrate how we can debug a pass using the
`PrintIR` and tracing.
